# SLEAP Distance Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys
from collections import defaultdict

In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import sleap.process_pose

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

# Functions

In [8]:
def combine_dicts(dicts):
    """
    Combine lists from multiple dictionaries that share the same key.

    This function takes a list of dictionaries where each dictionary's values are lists of numbers.
    It merges these lists for each corresponding key across all dictionaries, producing a single
    dictionary where each key has a combined list of all numbers from the input dictionaries.

    Parameters:
        dicts (list of dict): A list of dictionaries with values as lists of numbers.

    Returns:
        dict: A dictionary with keys from the input dictionaries and values as merged lists 
        of numbers from all corresponding input dictionary values.
    """
    combined = defaultdict(list)
    for dictionary in dicts:
        for key, value in dictionary.items():
            combined[key].extend(value)
    
    return dict(combined)


# Example usage
list_of_dicts = [
    {'a': [1, 2], 'b': [3, 4]},
    {'a': [5], 'b': [6, 7]},
    {'a': [8, 9], 'c': [10]}
]

combined_dict = combine_dicts(list_of_dicts)
print(combined_dict)

{'a': [1, 2, 5, 8, 9], 'b': [3, 4, 6, 7], 'c': [10]}


In [9]:
def find_consecutive_ranges(numbers, min_length=1):
    """
    Finds the start and end indices for consecutive ranges of each number in a list where the range meets a minimum length.

    This function iterates through a list of integers and identifies ranges where the same integer appears consecutively
    and the length of this sequence meets or exceeds the specified minimum length.

    Parameters:
        numbers (list): A list of integers to analyze for consecutive ranges.
        min_length (int): The minimum length of a range for it to be included in the results.

    Returns:
        dict: A dictionary with integers as keys and a list of tuples (start, end) as values,
              where each tuple represents the start and end indices (inclusive) of consecutive ranges
              for that integer. Only ranges that meet or exceed the minimum length are included.
    """
    ranges = {}
    n = len(numbers)
    if n == 0:
        return ranges
    
    start = 0
    current = numbers[0]

    for i in range(1, n):
        if numbers[i] != current:
            if (i - start) >= min_length:
                if current not in ranges:
                    ranges[current] = []
                ranges[current].append((start, i - 1))
            current = numbers[i]
            start = i

    # Handle the last range
    if (n - start) >= min_length:
        if current not in ranges:
            ranges[current] = []
        ranges[current].append((start, n - 1))

    return ranges

# Example usage:
numbers = [1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1]
print(find_consecutive_ranges(numbers, min_length=3))

{2: [(2, 4)], 3: [(5, 8)]}


In [10]:
def update_tuples_in_dict(original_dict, reference_list):
    """
    Updates the values in the tuples within a dictionary by replacing indices with corresponding values from a reference list.
    
    This function iterates through each key-value pair in the original dictionary. Each value is expected to be a list of tuples,
    where each tuple contains indices. These indices are used to fetch corresponding values from the reference list, creating new tuples.
    
    Parameters:
        original_dict (dict): Dictionary whose values are lists of tuples. Each tuple consists of indices into the reference_list.
        reference_list (list): List of elements that are referenced by the indices in the tuples of the original_dict.
        
    Returns:
        dict: A dictionary with the same keys as original_dict but with tuples transformed to contain elements from reference_list
              based on the indices in the original tuples.
    """
    # Create a new dictionary to store the updated key-value pairs
    new_dict = {}
    for key, list_of_tuples in original_dict.items():
        # Process each tuple in the list associated with the current key
        updated_tuples = [
            tuple(reference_list[idx] for idx in tup) for tup in list_of_tuples
        ]
        new_dict[key] = updated_tuples
    
    return new_dict

# Example usage:
original_dict = {
    'a': [(0, 1), (2, 3)],
    'b': [(1, 3), (0, 2)]
}
reference_list = ['alpha', 'beta', 'gamma', 'delta']

updated_dict = update_tuples_in_dict(original_dict, reference_list)
print(updated_dict)

{'a': [('alpha', 'beta'), ('gamma', 'delta')], 'b': [('beta', 'delta'), ('alpha', 'gamma')]}


## Inputs & Data

Explanation of each input and where it comes from.

In [11]:
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [12]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_01_lfp_traces_and_frames.pkl".format(OUTPUT_PREFIX))



In [13]:
# SLEAP_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")
SLEAP_DF = pd.read_pickle("../2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")

In [14]:
SLEAP_DF.head()

,video_name,current_subject,10s_after_tone_frame,10s_before_tone_frame,agent_locations,agent_nose,agent_nose_to_reward_port,agent_tail_base,agent_thorax,agent_thorax_velocity,...,experiment,first_timestamp,last_timestamp,reward_port,session_dir,sleap_name,start_frame,stop_frame,tracked_subject,video_id
0,20240317_151922_long_comp_subj_3-1_and_3-3.1,3.1,"[1686, 1686, 1686, 1686, 1686, 1686, 1686, 168...","[1086, 1086, 1086, 1086, 1086, 1086, 1086, 108...","[[[47.329585864054316, 13.395104517336417], [4...","[[43.139588993359475, 13.385259646809214], [43...","[29.850004, 29.948483, 30.033619, 30.119339, 3...","[[54.49241922411595, 12.722769593270066], [54....","[[51.04336785027878, 11.609328010205985], [51....","[1.75459, 0.8485628, 0.20676309, -0.17881195, ...",...,long_comp,2415571.0,48236575.0,"[60.41313477701829, 37.56463736381247]",20240317_151922_long_comp_subj_3-1_and_3-3,20240317_151922_long_comp_subj_3-1_and_3-3.1.f...,1,45736,"[3.1, 3.3]",0
1,20240317_151922_long_comp_subj_3-1_and_3-3.1,3.3,"[1686, 1686, 1686, 1686, 1686, 1686, 1686, 168...","[1086, 1086, 1086, 1086, 1086, 1086, 1086, 108...","[[[61.390550187751444, 36.122362967315595], [5...","[[60.14673983110819, 38.0809500426292], [60.14...","[0.7214881, 0.70931464, 0.7049949, 0.705891, 0...","[[65.86894804065032, 31.8436244490543], [65.86...","[[64.11036004541992, 34.137273635147906], [64....","[0.11019329, 0.08970148, 0.09392373, 0.1159680...",...,long_comp,2415571.0,48236575.0,"[60.41313477701829, 37.56463736381247]",20240317_151922_long_comp_subj_3-1_and_3-3,20240317_151922_long_comp_subj_3-1_and_3-3.1.f...,1,45736,"[3.1, 3.3]",0
2,20240317_172017_long_comp_subj_4-2_and_4-3.1,4.2,"[1701, 1701, 1701, 1701, 1701, 1701, 1701, 170...","[1100, 1100, 1100, 1100, 1100, 1100, 1100, 110...","[[[61.75545079959451, 33.21821636449367], [59....","[[60.79884848875681, 35.963595797483414], [60....","[1.2679133, 1.1848546, 1.0835545, 1.0503241, 1...","[[56.32709801682682, 29.966639024342353], [56....","[[57.3449526902064, 32.078917344899914], [57.5...","[0.28695866, 0.29812592, 0.3080636, 0.34382957...",...,long_comp,2020598.0,61007363.0,"[60.1935112853622, 35.25163617313764]",20240317_172017_long_comp_subj_4-2_and_4-3,20240317_172017_long_comp_subj_4-2_and_4-3.1.f...,1,58878,"[4.2, 4.3]",1
3,20240317_172017_long_comp_subj_4-2_and_4-3.1,4.3,"[1701, 1701, 1701, 1701, 1701, 1701, 1701, 170...","[1100, 1100, 1100, 1100, 1100, 1100, 1100, 110...","[[[67.1750209289574, 9.986812953490931], [69.1...","[[65.23110812441777, 8.220414676017084], [62.5...","[28.389486, 28.209822, 28.007545, 27.813864, 2...","[[72.8621368084351, 13.53665765315761], [72.41...","[[72.18478668206376, 10.328978616940205], [71....","[21.16527, 21.272135, 21.302916, 21.162853, 20...",...,long_comp,2020598.0,61007363.0,"[60.1935112853622, 35.25163617313764]",20240317_172017_long_comp_subj_4-2_and_4-3,20240317_172017_long_comp_subj_4-2_and_4-3.1.f...,1,58878,"[4.2, 4.3]",1
4,20240318_143819_long_comp_subj_3-3_and_3-4.1,3.3,"[1733, 1733, 1733, 1733, 1733, 1733, 1733, 173...","[1134, 1134, 1134, 1134, 1134, 1134, 1134, 113...","[[[61.29948457574409, 34.98487547102027], [61....","[[58.01061366578415, 33.98924535992776], [58.0...","[3.077167, 3.613538, 4.1945314, 4.827455, 5.53...","[[68.06216599593583, 32.37245454635553], [67.8...","[[65.57930503887175, 33.84569047638807], [65.3...","[8.288751, 9.374064, 10.581271, 11.979093, 13....",...,long_comp,1938832.0,60762064.0,"[60.56358180384908, 36.19153363328459]",20240318_143819_long_comp_subj_3-3_and_3-4,20240318_143819_long_comp_subj_3-3_and_3-4.1.f...,1,58752,"[3.3, 3.4]",2


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
FULL_LFP_TRACES_PKL = "{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

## Merging the dataframes based on shared video name

In [17]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [18]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132]

In [19]:
SLEAP_DF["video_name"] = SLEAP_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [20]:
SLEAP_DF["tone_start_timestamp"]

0     [1287955.0, 1287955.0, 1287955.0, 1287955.0, 1...
1     [1287955.0, 1287955.0, 1287955.0, 1287955.0, 1...
2     [1302635.0, 1302635.0, 1302635.0, 1302635.0, 1...
3     [1302635.0, 1302635.0, 1302635.0, 1302635.0, 1...
4     [1300498.0, 1300498.0, 1300498.0, 1300498.0, 1...
5     [1300498.0, 1300498.0, 1300498.0, 1300498.0, 1...
6     [1378134.0, 1378134.0, 1378134.0, 1378134.0, 1...
7     [1378134.0, 1378134.0, 1378134.0, 1378134.0, 1...
8     [1543730.0, 1543730.0, 1543730.0, 1543730.0, 1...
9     [1543730.0, 1543730.0, 1543730.0, 1543730.0, 1...
10    [1362950.0, 1362950.0, 1362950.0, 1362950.0, 1...
11    [1362950.0, 1362950.0, 1362950.0, 1362950.0, 1...
12    [34276299.0, 34276299.0, 34276299.0, 34276299....
13    [34276299.0, 34276299.0, 34276299.0, 34276299....
14    [34280576.0, 34280576.0, 34280576.0, 34280576....
15    [34280576.0, 34280576.0, 34280576.0, 34280576....
16    [1305657.0, 1305657.0, 1305657.0, 1305657.0, 1...
17    [1305657.0, 1305657.0, 1305657.0, 1305657.

In [21]:
cols_to_use = LFP_SPECTRAL_DF.columns.difference(SLEAP_DF.columns)
merged_LFP_SPECTRAL_DF = pd.merge(SLEAP_DF, LFP_SPECTRAL_DF[list(cols_to_use) + ["video_name", "current_subject"]], on=["video_name", "current_subject"], how="inner")

In [22]:
merged_LFP_SPECTRAL_DF.head()

,video_name,current_subject,10s_after_tone_frame,10s_before_tone_frame,agent_locations,agent_nose,agent_nose_to_reward_port,agent_tail_base,agent_thorax,agent_thorax_velocity,...,vHPC_gamma_band,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps
0,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.1,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[54.047201234350496, 32.48462136905939], [52...","[[57.31849083590113, 33.645587310504695], [56....","[4.960474, 5.34718, 5.802983, 6.407976, 7.1417...","[[46.630318040576064, 30.523901100123272], [46...","[[48.61806735273943, 32.66608744832453], [48.1...","[7.0735817, 9.794592, 12.977724, 16.4886, 20.1...",...,"[-0.07481611, -0.11126414, -0.14154938, -0.162...","[2.1274095, 2.4930122, 2.7189336, 3.0126748, -...","[65.715, 96.329994, 95.159996, 77.61, 112.905,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[0.8526041, 0.89195466, 0.93002737, 0.96665066...","[-1.0124702, -0.7600434, -0.73781365, -0.59900...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831..."
1,20240320_142408_alone_comp_subj_3-1_and_3-3.1,3.3,"[34640, 34640, 34640, 34640, 34640, 34640, 346...","[34040, 34040, 34040, 34040, 34040, 34040, 340...","[[[62.81573570925102, 33.49031610207749], [59....","[[60.5507800219845, 35.26465151447588], [60.78...","[0.53517926, 0.50113046, 0.48567954, 0.4737239...","[[66.06126838242345, 28.880162680346388], [66....","[[64.05699608055423, 31.17933144091776], [64.0...","[0.07300971, 0.08296164, 0.08879429, 0.0909930...",...,"[0.055602852, -0.017465636, -0.09318138, -0.16...","[1.1501461, 1.6517024, 1.9768442, 2.2265384, 2...","[-100.619995, -102.96, -129.87, -165.75, -191....","[[2.5391848, 1.5146569, 1.0411696, 1.5689967, ...","[[0.001042456, 0.009847558, 0.012313993, 0.009...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[-0.347932, -0.33184236, -0.31452566, -0.29600...","[2.6819186, -2.904363, -2.827914, -2.5523405, ...","[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831..."
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[56.40002042465876, 33.47263135770957], [55....","[[60.438020774390075, 35.57292836881663], [60....","[1.4890755, 1.1949695, 0.93336076, 0.738537, 0...","[[48.80137642581422, 31.16922356981376], [49.0...","[[51.35546426976598, 33.93773101458849], [52.3...","[10.536535, 10.738277, 10.965685, 11.176429, 1...",...,"[0.45194888, 0.3116872, 0.14765647, -0.0319272...","[0.023950227, 0.96471804, 1.2889154, 1.6205887...","[-74.685, -113.49, -156.78, -194.21999, -266.1...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.0013803464, 0.0079310145, 0.009366615, 0.0...","[0.00366902, nan, nan, 0.002184509, 0.00216185...","[0.046842232, nan, nan, 0.023685867, 0.0444350...","[-0.37066078, -0.35752764, -0.3432352, -0.3277...","[2.7155704, -2.8761566, -2.806554, -2.548516, ...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831..."
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34642, 34642, 34642, 34642, 34642, 34642, 346...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[[[61.39644296346538, 31.82081289424359], [58....","[[62.13302320168461, 34.13495024512682], [61.3...","[2.1410906, 2.5980964, 3.2073143, 3.9916637, 4...","[[50.83958858240558, 27.427984293098355], [50....","[[54.225662928143386, 30.269937128249303], [54...","[5.105928, 5.1050973, 5.082959, 5.072279, 5.09...",...,"[-0.065887734, -0.10607955, -0.14392401, -0.17...","[2.0565047, 2.3807256, 2.6063833, 2.87997, 3.1...","[53.234997, 80.145, 106.079994, 139.42499, 206...",

## Getting the timestamps of each cluster frame

In [23]:
merged_LFP_SPECTRAL_DF["frame_index"].iloc[0]

array([34040, 34041, 34042, ..., 61586, 61587, 61588])

In [24]:
merged_LFP_SPECTRAL_DF["video_timestamps"].iloc[0]

array([    1384,     2770,     4156, ..., 63592009, 63593395, 63594781],
      dtype=int32)

In [25]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"] = merged_LFP_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["frame_index"]], axis=1)


In [26]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].head()

0    [34077145, 34077145, 34078531, 34079917, 34079...
1    [34077145, 34077145, 34078531, 34079917, 34079...
2    [34079916, 34081302, 34082688, 34084074, 34084...
3    [34079916, 34081302, 34082688, 34084074, 34084...
4    [34103476, 34104862, 34106248, 34106248, 34107...
Name: cluster_timestamp, dtype: object

- Checking if the cluster timestamp shape matches the actual number of clusters

In [27]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].apply(lambda x: x.shape).head()

0    (12589,)
1    (12589,)
2    (12593,)
3    (12593,)
4    (12591,)
Name: cluster_timestamp, dtype: object

In [28]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].head()

0    [0, 0, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [0, 0, 0, 0, 0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
2    [7, 7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 2, ...
3    [7, 7, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 2, ...
4    [5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...
Name: kmeans_cluster, dtype: object

In [29]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].apply(lambda x: x.shape).head()

0    (12589,)
1    (12589,)
2    (12593,)
3    (12593,)
4    (12591,)
Name: kmeans_cluster, dtype: object

In [30]:
FULL_LFP_TRACES_PKL

'rce_pilot_3_alone_comp_04_spectral_and_sleap.pkl'

In [31]:
list(SLEAP_DF.columns)

['video_name',
 'current_subject',
 '10s_after_tone_frame',
 '10s_before_tone_frame',
 'agent_locations',
 'agent_nose',
 'agent_nose_to_reward_port',
 'agent_tail_base',
 'agent_thorax',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_agent_nose_to_reward_port',
 'closebool_subject_nose_to_reward_port',
 'clusterable_embedding_x',
 'clusterable_embedding_y',
 'color',
 'comp_id',
 'competition_closeness',
 'competitiveness',
 'condition ',
 'frame_index',
 'get_reward_frame',
 'kmeans_cluster',
 'manual_cluster_id',
 'movingbool_agent_thorax_velocity',
 'movingbool_subject_thorax_velocity',
 'nose_to_reward_port_diff',
 'nose_to_reward_port_sum',
 'nose_to_tail_diff',
 'nose_to_tail_sum',
 'notes',
 'out_reward_frame',
 'standard_embedding_x',
 'standard_embedding_y',
 'subject_locations',
 'subject_nose',
 'subject_nose_to_reward_port',
 'subject_tail_base',
 'subject_thorax',
 'subject_thorax_to_agent_thorax',
 'subject_thorax_velocity',
 'subject_to_reward_port

In [32]:
merged_LFP_SPECTRAL_DF["recording"].unique()

array(['20240320_142408_alone_comp_subj_3-1_t6b6_merged',
       '20240320_142408_alone_comp_subj_3-3_t5b5_merged',
       '20240320_171038_alone_comp_subj_4-2_t6b6_merged',
       '20240320_171038_alone_comp_subj_4-3_t5b5_merged',
       '20240322_120625_alone_comp_subj_3-3_t6b6_merged',
       '20240322_120625_alone_comp_subj_3-4_t5b5_merged',
       '20240322_160946_alone_comp_subj_4-3_t6b6_merged',
       '20240322_160946_alone_comp_subj_4-4_t5b5_merged',
       '20240323_122227_alone_comp_subj_5-2_t6b6_merged',
       '20240323_122227_alone_comp_subj_5-3_t5b5_merged',
       '20240323_144517_alone_comp_subj_3-1_t5b5_merged',
       '20240323_144517_alone_comp_subj_3-4_t6b6_merged',
       '20240323_165815_alone_comp_subj_4-2_t5b5_merged',
       '20240323_165815_alone_comp_subj_4-4_t6b6_merged'], dtype=object)

In [33]:
merged_LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))


In [34]:
merged_LFP_SPECTRAL_DF.shape

(14, 220)

In [35]:
raise ValueError()

ValueError: 